# Running a Multi Objective Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a multi objective optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib
import shutil

from IPython.display import Code

import boa
from boa.scripts.moo import Wrapper

[WARNING 07-13 14:31:07] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [2]:
# Remove old runs to have a clean slate for this example
old_runs = pathlib.Path().resolve().glob("moo_run*")
for path in old_runs:
    shutil.rmtree(path, ignore_errors=True)

## Loading the MOO Config File

In [3]:
config_path = pathlib.Path().resolve().parent.parent / "boa/scripts/moo.yaml"

Here we can see what the configuration file looks like

In [4]:
Code(config_path)

# MultiObjective Optimization config
optimization_options:
    objective_options:
        objective_thresholds:
            - branin >= -18.0
            - currin >= -6.0
        objectives:
            - name: branin
              lower_is_better: False
              noise_sd: 0
            - name: currin
              lower_is_better: False
              noise_sd: 0

    experiment:
        name: "moo_run"
    trials: 50

parameters:
    x0:
        type: range
        bounds: [0, 1]
        value_type: float
    x1:
        type: range
        bounds: [0, 1]
        value_type: float

In [5]:
config = boa.load_jsonlike(config_path)

## The Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [6]:
Code(Wrapper.path())

from pathlib import Path

import torch

from boa.controller import Controller
from boa.metrics.synthetic_funcs import get_synth_func
from boa.utils import torch_device
from boa.wrappers.base_wrapper import BaseWrapper

tkwargs = {
    "device": torch_device(),
}
Problem = get_synth_func("BraninCurrin")

problem = Problem(negate=True).to(**tkwargs)


class Wrapper(BaseWrapper):
    def run_model(self, trial) -> None:
        pass

    def set_trial_status(self, trial) -> None:
        trial.mark_completed()

    def fetch_trial_data(self, trial, metric_properties, metric_name, *args, **kwargs):
        evaluation = problem(torch.tensor([trial.arm.parameters["x0"], trial.arm.parameters["x1"]]))
        a = float(evaluation[0])
        b = float(evaluation[1])
        return {"branin": a, "currin": b}


def main():
    config_path = Path(__file__).resolve().parent / "moo.yaml"
    wrapper = Wrapper(config_path=config_path)
    controller = Controller(wrapper=wrapper)
    controller.initialize_scheduler()
    return controller.run()


if __name__ == "__main__":
    main()

## The Setup

In [7]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 07-13 14:31:09] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='x0', parameter_type=FLOAT, range=[0.0, 1.0]), RangeParameter(name='x1', parameter_type=FLOAT, range=[0.0, 1.0])], parameter_constraints=[]).
[INFO 07-13 14:31:09] ax.modelbridge.dispatch_utils: Using Models.MOO since there are more ordered parameters than there are categories for the unordered categorical parameters.
[INFO 07-13 14:31:09] ax.modelbridge.dispatch_utils: Calculating the number of remaining initialization trials based on num_initialization_trials=None max_initialization_trials=None num_tunable_parameters=2 num_trials=None use_batch_trials=False
[INFO 07-13 14:31:09] ax.modelbridge.dispatch_utils: calculated num_initialization_trials=5
[INFO 07-13 14:31:09] ax.modelbridge.dispatch_utils: num_completed_initialization_trials=0 num_remaining_initialization_trials=5
[INFO 07-13 14:31:09] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation s

(Scheduler(experiment=Experiment(moo_run), generation_strategy=GenerationStrategy(name='Sobol+MOO', steps=[Sobol for 5 trials, MOO for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <boa.scripts.moo.Wrapper at 0x1691f7fa0>)

## Start our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [8]:
scheduler = controller.run()

[INFO 2023-07-13 14:31:09,060 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/moo_run_20230713T143109
Start Time: 20230713T143109
Version: 0.8.7.dev4+gae30cf2.d20230713

##############################################

[INFO 07-13 14:31:09] Scheduler: Running trials [0]...
[INFO 07-13 14:31:10] Scheduler: Running trials [1]...
[INFO 07-13 14:31:11] Scheduler: Running trials [2]...
[INFO 07-13 14:31:12] Scheduler: Running trials [3]...
[INFO 07-13 14:31:13] Scheduler: Running trials [4]...
[INFO 07-13 14:31:14] Scheduler: Generated all trials that can be generated currently. Model requires more data to generate more trials.
[INFO 07-13 14:31:14] Scheduler: Retrieved COMPLETED trials: 0 - 4.
[INFO 07-13 14:31:14] Scheduler: Fetching data for trials: 0 - 4.
[ERROR 2023-07-13 14:31:14,208 MainProcess] boa: Object <boa.scripts.moo.Wrapper object at 0x1691f7fa0> pas

## Get the Best Trial

`best_fitted_trials` uses the data to do a fitting from all trials and with the noise levels you provided (or if no noise levels was provided, it assumed an unknown level of noise and inferred the noise level from the trial runs)

In [9]:
trial = scheduler.best_fitted_trials()
trial

[INFO 07-13 14:32:35] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.


{5: {'params': {'x0': 0.058607747184593696, 'x1': 1.0},
  'means': {'branin': -4.7050441454830025, 'currin': -3.3953789314264324},
  'cov_matrix': {'branin': {'branin': 0.00013756326699888865, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 1.841244830810778e-06}}},
 8: {'params': {'x0': 0.0, 'x1': 1.0},
  'means': {'branin': -17.506033432938764, 'currin': -1.180053704082634},
  'cov_matrix': {'branin': {'branin': 0.00013546227963810216, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 2.3068115437200997e-06}}},
 11: {'params': {'x0': 0.02788589200513968, 'x1': 1.0},
  'means': {'branin': -9.756586720344997, 'currin': -2.3002794474233026},
  'cov_matrix': {'branin': {'branin': 6.72758481420603e-05, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 9.92174240095971e-07}}},
 14: {'params': {'x0': 0.01337350973969159, 'x1': 1.0},
  'means': {'branin': -13.439514149380575, 'currin': -1.7253909772132952},
  'cov_matrix': {'branin': {'branin': 6.644393339241738e-05, 'currin

if you need the exact points of the best trial, maybe because you need the trial number of the best trial to plot results, or for any other reason, `best_raw_trails` does not do any fitting

In [10]:
trial = scheduler.best_raw_trials()
trial

[INFO 07-13 14:32:35] ax.modelbridge.torch: The observations are identical to the last set of observations used to fit the model. Skipping model fitting.


{5: {'params': {'x0': 0.058607747184593696, 'x1': 1.0},
  'means': {'branin': -4.707081317901611, 'currin': -3.395252227783203},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 8: {'params': {'x0': 0.0, 'x1': 1.0},
  'means': {'branin': -17.508296966552734, 'currin': -1.180408000946045},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 11: {'params': {'x0': 0.02788589200513968, 'x1': 1.0},
  'means': {'branin': -9.756551742553711, 'currin': -2.300353765487671},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 14: {'params': {'x0': 0.01337350973969159, 'x1': 1.0},
  'means': {'branin': -13.43986701965332, 'currin': -1.7252644300460815},
  'cov_matrix': {'branin': {'branin': 0.0, 'currin': 0.0},
   'currin': {'branin': 0.0, 'currin': 0.0}}},
 15: {'params': {'x0': 0.09760968515294619, 'x1': 0.9033515154200571},
  'm

## Output All Trials

In [11]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,branin,currin,is_feasible,x0,x1
0,0,0_0,COMPLETED,Sobol,-109.987946,-4.971244,False,0.856628,0.764265
1,1,1_0,COMPLETED,Sobol,-2.655920,-7.005593,False,0.157682,0.671000
2,2,2_0,COMPLETED,Sobol,-35.314487,-12.795367,False,0.357816,0.092607
3,3,3_0,COMPLETED,Sobol,-51.288143,-2.498643,False,0.016990,0.670656
4,4,4_0,COMPLETED,Sobol,-0.992895,-10.797516,False,0.562803,0.163890
5,5,5_0,COMPLETED,MOO,-4.707081,-3.395252,True,0.058608,1.000000
6,6,6_0,COMPLETED,MOO,-10.960894,-10.179487,False,1.000000,0.000000
7,7,7_0,COMPLETED,MOO,-55.937115,-5.405636,False,0.243453,1.000000
8,8,8_0,COMPLETED,MOO,-17.508297,-1.180408,True,0.000000,1.000000
9,9,9_0,COMPLETED,MOO,-26.455505,-1.280036,False,0.000000,0.898782
